In [2]:
from openai import OpenAI

from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass
from enum import Enum
import json
import logging
from abc import ABC, abstractmethod

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class AgentType(Enum):
    MAIN = "main"
    FILTER = "filter"
    ACTION = "action"

@dataclass
class ChatMessage:
    role: str  # 'user', 'assistant', 'system'
    content: str
    agent_type: Optional[AgentType] = None
    metadata: Optional[Dict[str, Any]] = None

@dataclass
class HandoffDecision:
    should_handoff: bool
    target_agent: Optional[AgentType] = None
    reason: str = ""
    continue_conversation: bool = True


In [3]:
class ConversationMemory:
    """Manages conversation history and context across agents"""
    
    def __init__(self):
        self.messages: List[ChatMessage] = []
        self.current_agent: AgentType = AgentType.MAIN
        self.agent_context: Dict[AgentType, Dict[str, Any]] = {
            AgentType.MAIN: {},
            AgentType.FILTER: {},
            AgentType.ACTION: {}
        }
        self.handoff_history: List[Tuple[AgentType, AgentType, str]] = []
    
    def add_message(self, message: ChatMessage):
        """Add a message to conversation history"""
        self.messages.append(message)
        logger.info(f"Added message from {message.agent_type}: {message.content[:50]}...")
    
    def get_recent_messages(self, count: int = 10) -> List[ChatMessage]:
        """Get recent messages for context"""
        return self.messages[-count:] if len(self.messages) >= count else self.messages
    
    def get_agent_context(self, agent_type: AgentType) -> Dict[str, Any]:
        """Get context specific to an agent"""
        return self.agent_context.get(agent_type, {})
    
    def update_agent_context(self, agent_type: AgentType, context: Dict[str, Any]):
        """Update agent-specific context"""
        self.agent_context[agent_type].update(context)
    
    def record_handoff(self, from_agent: AgentType, to_agent: AgentType, reason: str):
        """Record handoff between agents"""
        self.handoff_history.append((from_agent, to_agent, reason))
        self.current_agent = to_agent
        logger.info(f"Handoff: {from_agent.value} -> {to_agent.value} ({reason})")

In [4]:
class BaseAgent(ABC):
    """Base class for all agents"""
    
    def __init__(self, agent_type: AgentType, openai_client:OpenAI):
        self.agent_type = agent_type
        self.client = openai_client
        self.system_prompt = self._get_system_prompt()
    
    @abstractmethod
    def _get_system_prompt(self) -> str:
        """Get the system prompt for this agent"""
        pass
    
    @abstractmethod
    def process_query(self, query: str, memory: ConversationMemory) -> Tuple[str, HandoffDecision]:
        """Process a query and return response with handoff decision"""
        pass
    
    def _call_openai(self, messages: List[Dict[str, str]], max_tokens: int = 1000) -> str:
        """Make a call to OpenAI API"""
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"OpenAI API call failed: {e}")
            return f"Error: Unable to process request - {str(e)}"

In [5]:
class MainAgent(BaseAgent):
    """Main orchestrator agent that manages handoffs"""
    
    def __init__(self, openai_client: OpenAI):
        super().__init__(AgentType.MAIN, openai_client)
    
    def _get_system_prompt(self) -> str:
        return """You are the Main Orchestrator Agent in a multi-agent system. Your responsibilities:

1. ANALYZE user queries to determine which specialized agent should handle them
2. DECIDE handoffs between Filter Agent (for filtering/search queries) and Action Agent (for actions like add_outreach, get_csv, add_marketo)
3. MAINTAIN conversation context and flow
4. HANDLE requests to switch between agents mid-conversation

Agent Capabilities:
- Filter Agent: Handles filtering, searching, querying data, creating filter criteria
- Action Agent: Handles actions like add_outreach, get_csv, add_marketo, data manipulation

Decision Rules:
- If query involves filtering, searching, or querying -> Filter Agent
- If query involves actions or data manipulation -> Action Agent  
- If query asks to switch agents or is ambiguous -> Stay with Main Agent for clarification
- If query continues previous conversation -> Consider context

Respond with JSON format:
{
    "handoff_needed": true/false,
    "target_agent": "filter"/"action"/null,
    "reason": "explanation",
    "response": "your response to user"
}"""
    
    def process_query(self, query: str, memory: ConversationMemory) -> Tuple[str, HandoffDecision]:
        """Analyze query and decide on handoffs"""
        
        # Build context from recent messages
        recent_messages = memory.get_recent_messages(5)
        context = "\n".join([f"{msg.role}: {msg.content}" for msg in recent_messages[-3:]])
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"""
                                Current context: {context}
                                
                                Current agent: {memory.current_agent.value}
                                
                                User query: {query}
                                
                                Analyze this query and decide if handoff is needed."""}
                                        ]
                                        
        response = self._call_openai(messages)
        
        try:
            # Try to parse JSON response
            result = json.loads(response)
            handoff_decision = HandoffDecision(
                should_handoff=result.get("handoff_needed", False),
                target_agent=AgentType(result["target_agent"]) if result.get("target_agent") else None,
                reason=result.get("reason", "")
            )
            return result.get("response", response), handoff_decision
        except (json.JSONDecodeError, KeyError, ValueError):
            # Fallback if JSON parsing fails
            return response, HandoffDecision(should_handoff=False, reason="Failed to parse decision")



In [6]:
class FilterAgent(BaseAgent):
    """Agent specialized in filtering and search operations"""
    
    def __init__(self, openai_client: OpenAI):
        super().__init__(AgentType.FILTER, openai_client)
    
    def _get_system_prompt(self) -> str:
        return """You are the Filter Agent specialized in creating filter queries and handling search operations.

                Your capabilities:
                - Create filter queries based on user requirements
                - Handle search and filtering requests
                - Process data querying needs
                - Generate SQL-like filters
                - Handle complex search criteria
                
                For each response, also analyze if the user's query requires:
                1. Continuation with filtering/search tasks -> stay with Filter Agent
                2. Action operations (add_outreach, get_csv, add_marketo) -> handoff to Action Agent
                3. General questions or agent switching -> handoff to Main Agent
                
                Respond with JSON format:
                {
                    "filter_response": "your detailed filter response",
                    "handoff_needed": true/false,
                    "target_agent": "main"/"action"/null,
                    "reason": "explanation if handoff needed"
                }"""
    
    def process_query(self, query: str, memory: ConversationMemory) -> Tuple[str, HandoffDecision]:
        """Process filtering queries"""
        
        # Get filter-specific context
        filter_context = memory.get_agent_context(AgentType.FILTER)
        recent_messages = memory.get_recent_messages(3)
        context = "\n".join([f"{msg.role}: {msg.content}" for msg in recent_messages])
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"""
            Previous context: {context}
            Filter context: {filter_context}
            
            User query: {query}
            
            Process this filtering request and determine if handoff is needed."""}
                    ]
        
        response = self._call_openai(messages)
        
        try:
            result = json.loads(response)
            
            # Update filter context with query results
            memory.update_agent_context(AgentType.FILTER, {
                "last_query": query,
                "last_response": result.get("filter_response", "")
            })
            
            handoff_decision = HandoffDecision(
                should_handoff=result.get("handoff_needed", False),
                target_agent=AgentType(result["target_agent"]) if result.get("target_agent") else None,
                reason=result.get("reason", "")
            )
            
            return result.get("filter_response", response), handoff_decision
        except (json.JSONDecodeError, KeyError, ValueError):
            return response, HandoffDecision(should_handoff=False, reason="Continuing with filter operations")

In [7]:
class ActionAgent(BaseAgent):
    """Agent specialized in action operations"""
    
    def __init__(self, openai_client: OpenAI):
        super().__init__(AgentType.ACTION, openai_client)
    
    def _get_system_prompt(self) -> str:
        return """You are the Action Agent specialized in performing actions like add_outreach, get_csv, add_marketo.

                Your capabilities:
                - add_outreach: Add outreach campaigns
                - get_csv: Retrieve and process CSV data  
                - add_marketo: Add Marketo integrations
                - Data manipulation and processing actions
                - Execute operational tasks
                
                For each response, analyze if the user's query requires:
                1. More action operations -> stay with Action Agent
                2. Filtering/search operations -> handoff to Filter Agent
                3. General questions or agent switching -> handoff to Main Agent
                
                Respond with JSON format:
                {
                    "action_response": "your detailed action response with results",
                    "actions_performed": ["list", "of", "actions"],
                    "handoff_needed": true/false,
                    "target_agent": "main"/"filter"/null,
                    "reason": "explanation if handoff needed"
                }"""
    
    def process_query(self, query: str, memory: ConversationMemory) -> Tuple[str, HandoffDecision]:
        """Process action queries"""
        
        # Get action-specific context
        action_context = memory.get_agent_context(AgentType.ACTION)
        recent_messages = memory.get_recent_messages(3)
        context = "\n".join([f"{msg.role}: {msg.content}" for msg in recent_messages])
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"""
                Previous context: {context}
                Action context: {action_context}
                
                User query: {query}
                
                Process this action request and determine if handoff is needed."""}
                        ]
                        
        response = self._call_openai(messages)
        
        try:
            result = json.loads(response)
            
            # Update action context
            memory.update_agent_context(AgentType.ACTION, {
                "last_query": query,
                "last_actions": result.get("actions_performed", []),
                "last_response": result.get("action_response", "")
            })
            
            handoff_decision = HandoffDecision(
                should_handoff=result.get("handoff_needed", False),
                target_agent=AgentType(result["target_agent"]) if result.get("target_agent") else None,
                reason=result.get("reason", "")
            )
            
            return result.get("action_response", response), handoff_decision
        except (json.JSONDecodeError, KeyError, ValueError):
            return response, HandoffDecision(should_handoff=False, reason="Continuing with action operations")


In [8]:
class MultiAgentSystem:
    """Main multi-agent system orchestrator"""
    
    def __init__(self, openai_api_key: str):
        self.client = OpenAI(api_key=openai_api_key)
        self.memory = ConversationMemory()
        
        # Initialize agents
        self.agents = {
            AgentType.MAIN: MainAgent(self.client),
            AgentType.FILTER: FilterAgent(self.client),
            AgentType.ACTION: ActionAgent(self.client)
        }
        
        logger.info("Multi-agent system initialized")
    
    def process_query(self, user_query: str) -> str:
        """Process user query through the multi-agent system"""
        
        # Add user message to memory
        user_message = ChatMessage(role="user", content=user_query)
        self.memory.add_message(user_message)
        
        current_agent = self.agents[self.memory.current_agent]
        logger.info(f"Processing query with {current_agent.agent_type.value} agent")
        
        try:
            # Process query with current agent
            response, handoff_decision = current_agent.process_query(user_query, self.memory)
            
            # Add agent response to memory
            agent_message = ChatMessage(
                role="assistant", 
                content=response,
                agent_type=current_agent.agent_type
            )
            self.memory.add_message(agent_message)
            
            # Handle handoff if needed
            if handoff_decision.should_handoff and handoff_decision.target_agent:
                self.memory.record_handoff(
                    current_agent.agent_type, 
                    handoff_decision.target_agent, 
                    handoff_decision.reason
                )
                
                # If handoff to a different agent, process with new agent
                if handoff_decision.target_agent != current_agent.agent_type:
                    next_agent = self.agents[handoff_decision.target_agent]
                    next_response, _ = next_agent.process_query(user_query, self.memory)
                    
                    # Add the new agent's response
                    next_message = ChatMessage(
                        role="assistant",
                        content=next_response,
                        agent_type=handoff_decision.target_agent
                    )
                    self.memory.add_message(next_message)
                    
                    response = f"{response}\n\n[Handed off to {handoff_decision.target_agent.value} agent]\n\n{next_response}"
            
            return response
            
        except Exception as e:
            logger.error(f"Error processing query: {e}")
            return f"I apologize, but I encountered an error processing your request: {str(e)}"
    
    def get_conversation_history(self) -> List[Dict[str, Any]]:
        """Get formatted conversation history"""
        return [
            {
                "role": msg.role,
                "content": msg.content,
                "agent": msg.agent_type.value if msg.agent_type else None
            }
            for msg in self.memory.messages
        ]
    
    def get_current_agent(self) -> str:
        """Get current active agent"""
        return self.memory.current_agent.value
    
    def reset_conversation(self):
        """Reset the conversation and memory"""
        self.memory = ConversationMemory()
        logger.info("Conversation reset")

In [10]:
def main():
    # Initialize the system
    openai_api_key = "your-openai-api-key-here"  # Replace with your actual API key
    system = MultiAgentSystem(openai_api_key=None)
    
    print("Multi-Agent System Started!")
    print("Available commands:")
    print("- Type your queries naturally")
    print("- Type 'history' to see conversation history")
    print("- Type 'current' to see current agent")
    print("- Type 'reset' to reset conversation")
    print("- Type 'quit' to exit")
    print("-" * 50)
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            break
        elif user_input.lower() == 'history':
            history = system.get_conversation_history()
            for entry in history[-5:]:  # Show last 5 entries
                print(f"{entry['role']} ({entry['agent']}): {entry['content'][:100]}...")
        elif user_input.lower() == 'current':
            print(f"Current agent: {system.get_current_agent()}")
        elif user_input.lower() == 'reset':
            system.reset_conversation()
            print("Conversation reset!")
        elif user_input:
            response = system.process_query(user_input)
            print(f"\nSystem: {response}")

if __name__ == "__main__":
    main()

INFO:__main__:Multi-agent system initialized


Multi-Agent System Started!
Available commands:
- Type your queries naturally
- Type 'history' to see conversation history
- Type 'current' to see current agent
- Type 'reset' to reset conversation
- Type 'quit' to exit
--------------------------------------------------



You:  hello


INFO:__main__:Added message from None: hello...
INFO:__main__:Processing query with main agent
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Added message from AgentType.MAIN: ```json
{
    "handoff_needed": false,
    "target...



System: ```json
{
    "handoff_needed": false,
    "target_agent": null,
    "reason": "The user is simply greeting, no specific task or query has been mentioned.",
    "response": "Hello! How can I assist you today? If you have any tasks or queries, feel free to let me know."
}
```


...


System: ```json
{
    "handoff_needed": true,
    "target_agent": "filter",
    "reason": "The user is requesting to filter contacts by name, which involves searching and querying data.",
    "response": "I will hand you over to the Filter Agent to assist you with filtering contacts by names containing 'John'."
}
```



You:  Filter contacts by name containing 'John


INFO:__main__:Added message from None: Filter contacts by name containing 'John...
INFO:__main__:Processing query with main agent
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Added message from AgentType.MAIN: ```json
{
    "handoff_needed": true,
    "target_...



System: ```json
{
    "handoff_needed": true,
    "target_agent": "filter",
    "reason": "The user is requesting to filter contacts by name, which involves searching and querying data.",
    "response": "I will hand you over to the Filter Agent to assist you with filtering contacts by names containing 'John'."
}
```



You:  quit


In [13]:
# Initialize system
system = MultiAgentSystem(None)

# Example queries that would trigger different agents:
system.process_query("Filter customers by location and age")  # -> Filter Agent
system.process_query("Add outreach campaign for new leads")  # -> Action Agent  
system.process_query("Now get the CSV data for those results")  # -> Action Agent
system.process_query("Filter that data by purchase history")  # -> Filter Agent

system.reset_conversation()

INFO:__main__:Multi-agent system initialized
INFO:__main__:Added message from None: Filter customers by location and age...
INFO:__main__:Processing query with main agent
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Added message from AgentType.MAIN: ```json
{
    "handoff_needed": true,
    "target_...
INFO:__main__:Added message from None: Add outreach campaign for new leads...
INFO:__main__:Processing query with main agent
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Added message from AgentType.MAIN: ```json
{
    "handoff_needed": true,
    "target_...
INFO:__main__:Added message from None: Now get the CSV data for those results...
INFO:__main__:Processing query with main agent
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Added message from AgentType.MAIN: ```json
{
    "handoff_needed": true,
    "targe

# Generic 

In [14]:
import openai
import json
import uuid
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum

class AgentType(Enum):
    MAIN = "main"
    SPECIALIZED = "specialized"

class HandoffDecision(Enum):
    CONTINUE = "continue"
    HANDOFF_TO_MAIN = "handoff_to_main"
    HANDOFF_TO_AGENT = "handoff_to_agent"
    DIRECT_RESPONSE = "direct_response"

@dataclass
class Message:
    content: str
    role: str
    timestamp: datetime = field(default_factory=datetime.now)
    agent_id: Optional[str] = None
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class AgentMemory:
    agent_interactions: Dict[str, List[Message]] = field(default_factory=dict)
    agent_capabilities: Dict[str, List[str]] = field(default_factory=dict)
    context_history: List[Dict[str, Any]] = field(default_factory=list)
    current_agent: Optional[str] = None
    handoff_count: int = 0

class BaseAgent(ABC):
    def __init__(self, agent_id: str, name: str, description: str, capabilities: List[str]):
        self.agent_id = agent_id
        self.name = name
        self.description = description
        self.capabilities = capabilities
        self.client = openai.OpenAI()  # Initialize OpenAI client
        
    @abstractmethod
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        pass
    
    @abstractmethod
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        pass

class MainAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            agent_id="main_agent",
            name="Main Orchestrator Agent",
            description="Main agent responsible for routing queries to appropriate specialized agents",
            capabilities=["routing", "orchestration", "context_management", "handoff_decision"]
        )
        
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        """Process query and decide on agent routing"""
        
        # Create system prompt for main agent
        system_prompt = f"""
        You are the Main Orchestrator Agent in a multi-agent system. Your responsibilities:
        1. Analyze user queries to determine which specialized agent should handle them
        2. Maintain context across agent handoffs
        3. Decide when to route queries to specific agents
        4. Handle agent-to-agent communication
        
        Available agents and their capabilities:
        {self._get_available_agents_info()}
        
        Current context: {len(context)} previous messages
        
        For the user query, provide a JSON response with:
        - "target_agent": agent_id to route to (or "main_agent" to handle yourself)
        - "reasoning": explanation of routing decision
        - "context_summary": brief summary of relevant context
        - "response": your response if handling the query yourself
        """
        
        # Prepare context for LLM
        context_messages = []
        for msg in context[-10:]:  # Last 10 messages for context
            context_messages.append({
                "role": msg.role,
                "content": f"[{msg.agent_id}] {msg.content}" if msg.agent_id else msg.content
            })
        
        messages = [
            {"role": "system", "content": system_prompt},
            *context_messages,
            {"role": "user", "content": query}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            return result
            
        except Exception as e:
            return {
                "target_agent": "main_agent",
                "reasoning": f"Error in processing: {str(e)}",
                "context_summary": "Error occurred",
                "response": "I encountered an error processing your request. Please try again."
            }
    
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        """Main agent always processes queries first"""
        return HandoffDecision.CONTINUE, None
    
    def _get_available_agents_info(self) -> str:
        """Get information about available agents"""
        # This would be populated by the MultiAgentSystem
        return "This will be populated by the system with available agents"

class SpecializedAgent(BaseAgent):
    def __init__(self, agent_id: str, name: str, description: str, capabilities: List[str], 
                 specialized_prompt: str = ""):
        super().__init__(agent_id, name, description, capabilities)
        self.specialized_prompt = specialized_prompt
        
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        """Process query with specialized knowledge"""
        
        system_prompt = f"""
        You are {self.name}: {self.description}
        
        Your capabilities: {', '.join(self.capabilities)}
        
        {self.specialized_prompt}
        
        Important: If the user's query is outside your expertise or they're asking for a different type of agent,
        indicate that you should handoff back to the main agent.
        
        Provide a JSON response with:
        - "response": your response to the user
        - "confidence": confidence level (0-1) in handling this query
        - "should_handoff": boolean indicating if you should handoff
        - "handoff_reason": reason for handoff if applicable
        """
        
        # Prepare context
        context_messages = []
        for msg in context[-5:]:  # Last 5 messages for context
            context_messages.append({
                "role": msg.role,
                "content": msg.content
            })
        
        messages = [
            {"role": "system", "content": system_prompt},
            *context_messages,
            {"role": "user", "content": query}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.5,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            return result
            
        except Exception as e:
            return {
                "response": f"I encountered an error processing your request: {str(e)}",
                "confidence": 0.0,
                "should_handoff": True,
                "handoff_reason": "Error occurred"
            }
    
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        """Determine if specialized agent should handoff"""
        result = self.process_query(query, context)
        
        if result.get("should_handoff", False):
            return HandoffDecision.HANDOFF_TO_MAIN, "main_agent"
        elif result.get("confidence", 0) < 0.3:
            return HandoffDecision.HANDOFF_TO_MAIN, "main_agent"
        else:
            return HandoffDecision.DIRECT_RESPONSE, None

class MultiAgentSystem:
    def __init__(self, openai_api_key: str):
        """Initialize the multi-agent system"""
        openai.api_key = openai_api_key
        
        self.agents: Dict[str, BaseAgent] = {}
        self.memory = AgentMemory()
        self.session_id = str(uuid.uuid4())
        
        # Initialize main agent
        self.main_agent = MainAgent()
        self.agents[self.main_agent.agent_id] = self.main_agent
        self.memory.current_agent = self.main_agent.agent_id
        
        # Update main agent with system reference
        self.main_agent._get_available_agents_info = self._get_agents_info
    
    def add_agent(self, agent: BaseAgent) -> None:
        """Add a specialized agent to the system"""
        self.agents[agent.agent_id] = agent
        self.memory.agent_capabilities[agent.agent_id] = agent.capabilities
        
    def _get_agents_info(self) -> str:
        """Get formatted information about all available agents"""
        info = []
        for agent_id, agent in self.agents.items():
            if agent_id != "main_agent":
                info.append(f"- {agent.name} ({agent_id}): {agent.description}")
                info.append(f"  Capabilities: {', '.join(agent.capabilities)}")
        return "\n".join(info)
    
    def process_user_query(self, query: str) -> Dict[str, Any]:
        """Process user query through the multi-agent system"""
        
        # Add user message to memory
        user_message = Message(content=query, role="user")
        self._add_message_to_memory(user_message)
        
        # Get current context
        context = self._get_context_messages()
        
        # Start with main agent decision
        current_agent_id = "main_agent"
        max_handoffs = 5
        handoff_count = 0
        
        while handoff_count < max_handoffs:
            current_agent = self.agents[current_agent_id]
            
            # Process query with current agent
            result = current_agent.process_query(query, context)
            
            # Check if we need to handoff
            handoff_decision, target_agent = current_agent.should_handoff(query, context)
            
            if handoff_decision == HandoffDecision.DIRECT_RESPONSE:
                # Agent can handle the query directly
                response_message = Message(
                    content=result.get("response", "No response provided"),
                    role="assistant",
                    agent_id=current_agent_id
                )
                self._add_message_to_memory(response_message)
                self.memory.current_agent = current_agent_id
                
                return {
                    "response": result.get("response", "No response provided"),
                    "agent_used": current_agent.name,
                    "agent_id": current_agent_id,
                    "handoff_count": handoff_count,
                    "confidence": result.get("confidence", 1.0),
                    "session_id": self.session_id
                }
            
            elif handoff_decision == HandoffDecision.HANDOFF_TO_MAIN:
                # Handoff back to main agent
                if current_agent_id == "main_agent":
                    # Already at main agent, provide response
                    response_message = Message(
                        content=result.get("response", "I'm not sure how to help with that."),
                        role="assistant",
                        agent_id=current_agent_id
                    )
                    self._add_message_to_memory(response_message)
                    
                    return {
                        "response": result.get("response", "I'm not sure how to help with that."),
                        "agent_used": current_agent.name,
                        "agent_id": current_agent_id,
                        "handoff_count": handoff_count,
                        "session_id": self.session_id
                    }
                else:
                    current_agent_id = "main_agent"
                    handoff_count += 1
                    continue
            
            elif handoff_decision == HandoffDecision.HANDOFF_TO_AGENT:
                # Handoff to specific agent
                if target_agent and target_agent in self.agents:
                    current_agent_id = target_agent
                    handoff_count += 1
                    continue
                else:
                    # Target agent not found, use main agent response
                    response_message = Message(
                        content=result.get("response", "Agent not found."),
                        role="assistant",
                        agent_id=current_agent_id
                    )
                    self._add_message_to_memory(response_message)
                    
                    return {
                        "response": result.get("response", "Agent not found."),
                        "agent_used": current_agent.name,
                        "agent_id": current_agent_id,
                        "handoff_count": handoff_count,
                        "session_id": self.session_id
                    }
            
            # For main agent, check if it wants to route to another agent
            if current_agent_id == "main_agent":
                target_agent_id = result.get("target_agent")
                if target_agent_id and target_agent_id != "main_agent" and target_agent_id in self.agents:
                    current_agent_id = target_agent_id
                    handoff_count += 1
                    continue
                else:
                    # Main agent handles the query itself
                    response_message = Message(
                        content=result.get("response", "How can I help you?"),
                        role="assistant",
                        agent_id=current_agent_id
                    )
                    self._add_message_to_memory(response_message)
                    
                    return {
                        "response": result.get("response", "How can I help you?"),
                        "agent_used": current_agent.name,
                        "agent_id": current_agent_id,
                        "handoff_count": handoff_count,
                        "reasoning": result.get("reasoning", ""),
                        "session_id": self.session_id
                    }
        
        # Max handoffs reached
        return {
            "response": "I've reached the maximum number of agent handoffs. Please try rephrasing your question.",
            "agent_used": "System",
            "agent_id": "system",
            "handoff_count": handoff_count,
            "session_id": self.session_id
        }
    
    def _add_message_to_memory(self, message: Message) -> None:
        """Add message to memory"""
        if message.agent_id:
            if message.agent_id not in self.memory.agent_interactions:
                self.memory.agent_interactions[message.agent_id] = []
            self.memory.agent_interactions[message.agent_id].append(message)
        
        # Add to context history
        self.memory.context_history.append({
            "message": message,
            "timestamp": message.timestamp,
            "agent_id": message.agent_id
        })
    
    def _get_context_messages(self) -> List[Message]:
        """Get recent context messages"""
        recent_messages = []
        for entry in self.memory.context_history[-20:]:  # Last 20 messages
            recent_messages.append(entry["message"])
        return recent_messages
    
    def get_memory_summary(self) -> Dict[str, Any]:
        """Get summary of current memory state"""
        return {
            "current_agent": self.memory.current_agent,
            "total_messages": len(self.memory.context_history),
            "agents_used": list(self.memory.agent_interactions.keys()),
            "handoff_count": self.memory.handoff_count,
            "session_id": self.session_id
        }
    
    def clear_memory(self) -> None:
        """Clear memory for new session"""
        self.memory = AgentMemory()
        self.memory.current_agent = "main_agent"
        self.session_id = str(uuid.uuid4())

# Example specialized agents
class CodeAssistantAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            agent_id="code_assistant",
            name="Code Assistant",
            description="Specialized in helping with programming, debugging, and code reviews",
            capabilities=["programming", "debugging", "code_review", "algorithm_design"],
            specialized_prompt="""
            You are an expert programming assistant. You can help with:
            - Writing and reviewing code in various languages
            - Debugging and troubleshooting
            - Algorithm design and optimization
            - Best practices and code architecture
            
            If asked about non-programming topics, suggest handoff to main agent.
            """
        )

class DataAnalystAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            agent_id="data_analyst",
            name="Data Analyst",
            description="Specialized in data analysis, statistics, and visualization",
            capabilities=["data_analysis", "statistics", "visualization", "machine_learning"],
            specialized_prompt="""
            You are an expert data analyst. You can help with:
            - Data analysis and interpretation
            - Statistical analysis and hypothesis testing
            - Data visualization recommendations
            - Machine learning model suggestions
            
            If asked about non-data topics, suggest handoff to main agent.
            """
        )

class WritingAssistantAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            agent_id="writing_assistant",
            name="Writing Assistant",
            description="Specialized in creative writing, editing, and content creation",
            capabilities=["creative_writing", "editing", "content_creation", "grammar_check"],
            specialized_prompt="""
            You are an expert writing assistant. You can help with:
            - Creative writing and storytelling
            - Editing and proofreading
            - Content creation for various formats
            - Grammar and style improvements
            
            If asked about technical or non-writing topics, suggest handoff to main agent.
            """
        )

# Example usage
if __name__ == "__main__":
    # Initialize the system
    system = MultiAgentSystem(openai_api_key="your-openai-api-key-here")
    
    # Add specialized agents
    system.add_agent(CodeAssistantAgent())
    system.add_agent(DataAnalystAgent())
    system.add_agent(WritingAssistantAgent())
    
    # Example interaction
    print("Multi-Agent System initialized!")
    print("Available agents:")
    for agent_id, agent in system.agents.items():
        print(f"- {agent.name} ({agent_id})")
    
    # Example queries
    example_queries = [
        "Help me write a Python function to sort a list",
        "I need help analyzing my sales data",
        "Can you help me write a creative story?",
        "What's the weather like today?",  # Should be handled by main agent
    ]
    
    for query in example_queries:
        print(f"\n--- Query: {query} ---")
        result = system.process_user_query(query)
        print(f"Agent: {result['agent_used']}")
        print(f"Response: {result['response']}")
        print(f"Handoffs: {result['handoff_count']}")

Multi-Agent System initialized!
Available agents:
- Main Orchestrator Agent (main_agent)
- Code Assistant (code_assistant)
- Data Analyst (data_analyst)
- Writing Assistant (writing_assistant)

--- Query: Help me write a Python function to sort a list ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Agent: Code Assistant
Response: Certainly! Here is a simple Python function that sorts a list using the built-in `sorted()` function:

```python

def sort_list(input_list):
    return sorted(input_list)

# Example usage:
my_list = [5, 2, 9, 1, 5, 6]
sorted_list = sort_list(my_list)
print(sorted_list)  # Output: [1, 2, 5, 5, 6, 9]
```

This function takes a list as input and returns a new list that is sorted in ascending order. The `sorted()` function is efficient and handles sorting internally. If you need to sort the list in place, you can use the `sort()` method of the list object itself.
Handoffs: 1

--- Query: I need help analyzing my sales data ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Agent: Data Analyst
Response: I'd be happy to help you analyze your sales data! To get started, could you provide more details about the data you have? For example, the type of data (e.g., sales figures, customer demographics), the format (e.g., CSV, Excel), and any specific questions or insights you are looking to gain from the analysis.
Handoffs: 1

--- Query: Can you help me write a creative story? ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Agent: Writing Assistant
Response: Of course! I'd love to help you write a creative story. To get started, could you provide some details or themes you'd like to explore? For example, do you have a particular genre in mind, such as fantasy, mystery, or romance? Or perhaps a setting or a character you want to develop? Let me know, and we can start crafting your story together!
Handoffs: 1

--- Query: What's the weather like today? ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Agent: Main Orchestrator Agent
Response: I'm unable to provide real-time weather updates. However, you can check the current weather by using a weather website or app like Weather.com or a weather service API if you're looking to integrate weather data into a project.
Handoffs: 0


In [15]:
import openai
import json
import uuid
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum

class AgentType(Enum):
    MAIN = "main"
    SPECIALIZED = "specialized"

class HandoffDecision(Enum):
    CONTINUE = "continue"
    HANDOFF_TO_MAIN = "handoff_to_main"
    HANDOFF_TO_AGENT = "handoff_to_agent"
    DIRECT_RESPONSE = "direct_response"

@dataclass
class Message:
    content: str
    role: str
    timestamp: datetime = field(default_factory=datetime.now)
    agent_id: Optional[str] = None
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class AgentMemory:
    agent_interactions: Dict[str, List[Message]] = field(default_factory=dict)
    agent_capabilities: Dict[str, List[str]] = field(default_factory=dict)
    context_history: List[Dict[str, Any]] = field(default_factory=list)
    current_agent: Optional[str] = None
    handoff_count: int = 0

class BaseAgent(ABC):
    def __init__(self, agent_id: str, name: str, description: str, capabilities: List[str]):
        self.agent_id = agent_id
        self.name = name
        self.description = description
        self.capabilities = capabilities
        self.client = openai.OpenAI()  # Initialize OpenAI client
        
    @abstractmethod
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        pass
    
    @abstractmethod
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        pass

class MainAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            agent_id="main_agent",
            name="Main Orchestrator Agent",
            description="Main agent responsible for routing queries to appropriate specialized agents",
            capabilities=["routing", "orchestration", "context_management", "handoff_decision"]
        )
        
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        """Process query and decide on agent routing"""
        
        # Create system prompt for main agent
        system_prompt = f"""
        You are the Main Orchestrator Agent in a multi-agent system. Your responsibilities:
        1. Analyze user queries to determine which specialized agent should handle them
        2. Maintain context across agent handoffs
        3. Decide when to route queries to specific agents
        4. Handle agent-to-agent communication
        
        Available agents and their capabilities:
        {self._get_available_agents_info()}
        
        Current context: {len(context)} previous messages
        
        For the user query, provide a JSON response with:
        - "target_agent": agent_id to route to (or "main_agent" to handle yourself)
        - "reasoning": explanation of routing decision
        - "context_summary": brief summary of relevant context
        - "response": your response if handling the query yourself
        """
        
        # Prepare context for LLM
        context_messages = []
        for msg in context[-10:]:  # Last 10 messages for context
            context_messages.append({
                "role": msg.role,
                "content": f"[{msg.agent_id}] {msg.content}" if msg.agent_id else msg.content
            })
        
        messages = [
            {"role": "system", "content": system_prompt},
            *context_messages,
            {"role": "user", "content": query}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            return result
            
        except Exception as e:
            return {
                "target_agent": "main_agent",
                "reasoning": f"Error in processing: {str(e)}",
                "context_summary": "Error occurred",
                "response": "I encountered an error processing your request. Please try again."
            }
    
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        """Main agent always processes queries first"""
        return HandoffDecision.CONTINUE, None
    
    def _get_available_agents_info(self) -> str:
        """Get information about available agents"""
        # This would be populated by the MultiAgentSystem
        return "This will be populated by the system with available agents"

class SpecializedAgent(BaseAgent):
    def __init__(self, agent_id: str, name: str, description: str, capabilities: List[str], 
                 specialized_prompt: str = ""):
        super().__init__(agent_id, name, description, capabilities)
        self.specialized_prompt = specialized_prompt
        
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        """Process query with specialized knowledge"""
        
        system_prompt = f"""
        You are {self.name}: {self.description}
        
        Your capabilities: {', '.join(self.capabilities)}
        
        {self.specialized_prompt}
        
        Important: If the user's query is outside your expertise or they're asking for a different type of agent,
        indicate that you should handoff back to the main agent.
        
        Provide a JSON response with:
        - "response": your response to the user
        - "confidence": confidence level (0-1) in handling this query
        - "should_handoff": boolean indicating if you should handoff
        - "handoff_reason": reason for handoff if applicable
        """
        
        # Prepare context
        context_messages = []
        for msg in context[-5:]:  # Last 5 messages for context
            context_messages.append({
                "role": msg.role,
                "content": msg.content
            })
        
        messages = [
            {"role": "system", "content": system_prompt},
            *context_messages,
            {"role": "user", "content": query}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.5,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            return result
            
        except Exception as e:
            return {
                "response": f"I encountered an error processing your request: {str(e)}",
                "confidence": 0.0,
                "should_handoff": True,
                "handoff_reason": "Error occurred"
            }
    
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        """Determine if specialized agent should handoff"""
        result = self.process_query(query, context)
        
        if result.get("should_handoff", False):
            return HandoffDecision.HANDOFF_TO_MAIN, "main_agent"
        elif result.get("confidence", 0) < 0.3:
            return HandoffDecision.HANDOFF_TO_MAIN, "main_agent"
        else:
            return HandoffDecision.DIRECT_RESPONSE, None

class MultiAgentSystem:
    def __init__(self, openai_api_key: str):
        """Initialize the multi-agent system"""
        openai.api_key = openai_api_key
        
        self.agents: Dict[str, BaseAgent] = {}
        self.memory = AgentMemory()
        self.session_id = str(uuid.uuid4())
        self.active_agent_id = None  # Track which agent is currently active
        
        # Initialize main agent
        self.main_agent = MainAgent()
        self.agents[self.main_agent.agent_id] = self.main_agent
        self.memory.current_agent = self.main_agent.agent_id
        self.active_agent_id = self.main_agent.agent_id
        
        # Update main agent with system reference
        self.main_agent._get_available_agents_info = self._get_agents_info
    
    def add_agent(self, agent: BaseAgent) -> None:
        """Add a specialized agent to the system"""
        self.agents[agent.agent_id] = agent
        self.memory.agent_capabilities[agent.agent_id] = agent.capabilities
        
    def _get_agents_info(self) -> str:
        """Get formatted information about all available agents"""
        info = []
        for agent_id, agent in self.agents.items():
            if agent_id != "main_agent":
                info.append(f"- {agent.name} ({agent_id}): {agent.description}")
                info.append(f"  Capabilities: {', '.join(agent.capabilities)}")
        return "\n".join(info)
    
    def process_user_query(self, query: str) -> Dict[str, Any]:
        """Process user query in continuous chat session with agent handoffs"""
        
        # Add user message to memory
        user_message = Message(content=query, role="user")
        self._add_message_to_memory(user_message)
        
        # Get current context
        context = self._get_context_messages()
        
        # Use the currently active agent (could be specialized agent from previous interaction)
        current_agent_id = self.active_agent_id
        current_agent = self.agents[current_agent_id]
        
        print(f"🤖 Current active agent: {current_agent.name}")
        
        # If we're with a specialized agent, first check if it can handle the query
        if current_agent_id != "main_agent":
            # Let specialized agent try to handle the query
            result = current_agent.process_query(query, context)
            handoff_decision, target_agent = current_agent.should_handoff(query, context)
            
            if handoff_decision == HandoffDecision.DIRECT_RESPONSE:
                # Specialized agent can handle it - direct interaction continues
                response_message = Message(
                    content=result.get("response", "No response provided"),
                    role="assistant",
                    agent_id=current_agent_id
                )
                self._add_message_to_memory(response_message)
                
                return {
                    "response": result.get("response", "No response provided"),
                    "agent_used": current_agent.name,
                    "agent_id": current_agent_id,
                    "handoff_occurred": False,
                    "confidence": result.get("confidence", 1.0),
                    "session_id": self.session_id,
                    "handoff_reason": None
                }
            
            elif handoff_decision == HandoffDecision.HANDOFF_TO_MAIN:
                # Specialized agent wants to handoff back to main agent
                print(f"🔄 {current_agent.name} handing off to Main Agent")
                print(f"📝 Handoff reason: {result.get('handoff_reason', 'Query outside expertise')}")
                
                # Switch to main agent
                self.active_agent_id = "main_agent"
                current_agent_id = "main_agent"
                current_agent = self.agents[current_agent_id]
                
                # Add handoff message to context
                handoff_message = Message(
                    content=f"[HANDOFF] {self.agents[self.memory.current_agent].name} is handing off to Main Agent. Reason: {result.get('handoff_reason', 'Query requires different expertise')}",
                    role="system",
                    agent_id="system"
                )
                self._add_message_to_memory(handoff_message)
        
        # Now process with main agent (either started here or handed off)
        if current_agent_id == "main_agent":
            result = current_agent.process_query(query, context)
            target_agent_id = result.get("target_agent")
            
            # Main agent decides routing
            if target_agent_id and target_agent_id != "main_agent" and target_agent_id in self.agents:
                # Main agent wants to handoff to specialized agent
                print(f"🎯 Main Agent routing to: {self.agents[target_agent_id].name}")
                print(f"💭 Reasoning: {result.get('reasoning', 'Best suited for this query')}")
                
                # Switch active agent
                self.active_agent_id = target_agent_id
                specialized_agent = self.agents[target_agent_id]
                
                # Add handoff message
                handoff_message = Message(
                    content=f"[HANDOFF] Main Agent routing to {specialized_agent.name}. User query: {query}",
                    role="system",
                    agent_id="system"
                )
                self._add_message_to_memory(handoff_message)
                
                # Get response from specialized agent
                specialized_context = self._get_context_messages()
                specialized_result = specialized_agent.process_query(query, specialized_context)
                
                # Store response
                response_message = Message(
                    content=specialized_result.get("response", "Hello! I'm now handling your request."),
                    role="assistant",
                    agent_id=target_agent_id
                )
                self._add_message_to_memory(response_message)
                self.memory.current_agent = target_agent_id
                
                return {
                    "response": specialized_result.get("response", "Hello! I'm now handling your request."),
                    "agent_used": specialized_agent.name,
                    "agent_id": target_agent_id,
                    "handoff_occurred": True,
                    "previous_agent": "Main Agent",
                    "handoff_reason": result.get("reasoning", "Specialized expertise needed"),
                    "confidence": specialized_result.get("confidence", 1.0),
                    "session_id": self.session_id
                }
            
            else:
                # Main agent handles the query itself
                response_message = Message(
                    content=result.get("response", "How can I help you?"),
                    role="assistant",
                    agent_id=current_agent_id
                )
                self._add_message_to_memory(response_message)
                self.memory.current_agent = current_agent_id
                
                return {
                    "response": result.get("response", "How can I help you?"),
                    "agent_used": current_agent.name,
                    "agent_id": current_agent_id,
                    "handoff_occurred": False,
                    "reasoning": result.get("reasoning", ""),
                    "session_id": self.session_id
                }
    
    def _add_message_to_memory(self, message: Message) -> None:
        """Add message to memory"""
        if message.agent_id:
            if message.agent_id not in self.memory.agent_interactions:
                self.memory.agent_interactions[message.agent_id] = []
            self.memory.agent_interactions[message.agent_id].append(message)
        
        # Add to context history
        self.memory.context_history.append({
            "message": message,
            "timestamp": message.timestamp,
            "agent_id": message.agent_id
        })
    
    def _get_context_messages(self) -> List[Message]:
        """Get recent context messages"""
        recent_messages = []
        for entry in self.memory.context_history[-20:]:  # Last 20 messages
            recent_messages.append(entry["message"])
        return recent_messages
    
    def get_memory_summary(self) -> Dict[str, Any]:
        """Get summary of current memory state"""
        return {
            "current_agent": self.memory.current_agent,
            "active_agent": self.active_agent_id,
            "active_agent_name": self.agents[self.active_agent_id].name if self.active_agent_id else None,
            "total_messages": len(self.memory.context_history),
            "agents_used": list(self.memory.agent_interactions.keys()),
            "handoff_count": self.memory.handoff_count,
            "session_id": self.session_id
        }
    
    def clear_memory(self) -> None:
        """Clear memory for new session"""
        self.memory = AgentMemory()
        self.memory.current_agent = "main_agent"
        self.active_agent_id = "main_agent"
        self.session_id = str(uuid.uuid4())
    
    def get_active_agent_info(self) -> Dict[str, Any]:
        """Get information about currently active agent"""
        if self.active_agent_id and self.active_agent_id in self.agents:
            agent = self.agents[self.active_agent_id]
            return {
                "agent_id": agent.agent_id,
                "name": agent.name,
                "description": agent.description,
                "capabilities": agent.capabilities,
                "is_main_agent": agent.agent_id == "main_agent"
            }
        return None

# Example specialized agents
class CodeAssistantAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            agent_id="code_assistant",
            name="Code Assistant",
            description="Specialized in helping with programming, debugging, and code reviews",
            capabilities=["programming", "debugging", "code_review", "algorithm_design"],
            specialized_prompt="""
            You are an expert programming assistant. You can help with:
            - Writing and reviewing code in various languages
            - Debugging and troubleshooting
            - Algorithm design and optimization
            - Best practices and code architecture
            
            If asked about non-programming topics, suggest handoff to main agent.
            """
        )

class DataAnalystAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            agent_id="data_analyst",
            name="Data Analyst",
            description="Specialized in data analysis, statistics, and visualization",
            capabilities=["data_analysis", "statistics", "visualization", "machine_learning"],
            specialized_prompt="""
            You are an expert data analyst. You can help with:
            - Data analysis and interpretation
            - Statistical analysis and hypothesis testing
            - Data visualization recommendations
            - Machine learning model suggestions
            
            If asked about non-data topics, suggest handoff to main agent.
            """
        )

class WritingAssistantAgent(SpecializedAgent):
    def __init__(self):
        super().__init__(
            agent_id="writing_assistant",
            name="Writing Assistant",
            description="Specialized in creative writing, editing, and content creation",
            capabilities=["creative_writing", "editing", "content_creation", "grammar_check"],
            specialized_prompt="""
            You are an expert writing assistant. You can help with:
            - Creative writing and storytelling
            - Editing and proofreading
            - Content creation for various formats
            - Grammar and style improvements
            
            If asked about technical or non-writing topics, suggest handoff to main agent.
            """
        )

# Example usage and conversation flow demonstration
if __name__ == "__main__":
    # Initialize the system
    system = MultiAgentSystem(openai_api_key="your-openai-api-key-here")
    
    # Add specialized agents
    system.add_agent(CodeAssistantAgent())
    system.add_agent(DataAnalystAgent())
    system.add_agent(WritingAssistantAgent())
    
    print("🚀 Multi-Agent System initialized!")
    print("Available agents:")
    for agent_id, agent in system.agents.items():
        print(f"- {agent.name} ({agent_id})")
    
    print("\n" + "="*50)
    print("CONVERSATION FLOW DEMONSTRATION")
    print("="*50)
    
    # Simulate a conversation flow
    conversation_flow = [
        "Help me write a Python function to calculate fibonacci numbers",  # Should route to Code Assistant
        "Can you optimize this for better performance?",  # Code Assistant should continue
        "Now help me write a story about a programmer",  # Code Assistant should handoff to Main Agent, then to Writing Assistant
        "Make it more dramatic and exciting",  # Writing Assistant should continue
        "What's the time complexity of the fibonacci function we discussed earlier?",  # Writing Assistant should handoff back to Main Agent, then to Code Assistant
        "Can you analyze some sales data for me?",  # Code Assistant should handoff to Main Agent, then to Data Analyst
    ]
    
    for i, query in enumerate(conversation_flow, 1):
        print(f"\n--- Step {i}: User Query ---")
        print(f"User: {query}")
        
        # Get current active agent before processing
        active_info = system.get_active_agent_info()
        print(f"🤖 Currently active: {active_info['name']}")
        
        # Process the query
        result = system.process_user_query(query)
        
        # Display results
        print(f"👨‍💼 Agent used: {result['agent_used']}")
        if result.get('handoff_occurred'):
            print(f"🔄 Handoff occurred: {result.get('previous_agent', 'Unknown')} → {result['agent_used']}")
            print(f"📝 Reason: {result.get('handoff_reason', 'Not specified')}")
        
        print(f"🤖 Response: {result['response'][:200]}{'...' if len(result['response']) > 200 else ''}")
        
        # Show memory summary
        memory = system.get_memory_summary()
        print(f"📊 Session info: {memory['total_messages']} messages, Active: {memory['active_agent_name']}")
        print("-" * 80)
    
    print("\n🎉 Conversation flow demonstration complete!")
    print("\nKey Features Demonstrated:")
    print("✅ Direct agent-to-user interaction")
    print("✅ Seamless handoffs between agents")
    print("✅ Context preservation across handoffs")
    print("✅ Intelligent routing based on query content")
    print("✅ Agents can continue conversations in their domain")
    print("✅ Automatic handoff when expertise is needed elsewhere")

🚀 Multi-Agent System initialized!
Available agents:
- Main Orchestrator Agent (main_agent)
- Code Assistant (code_assistant)
- Data Analyst (data_analyst)
- Writing Assistant (writing_assistant)

CONVERSATION FLOW DEMONSTRATION

--- Step 1: User Query ---
User: Help me write a Python function to calculate fibonacci numbers
🤖 Currently active: Main Orchestrator Agent
🤖 Current active agent: Main Orchestrator Agent


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🎯 Main Agent routing to: Code Assistant
💭 Reasoning: The user is requesting help with writing a Python function, which falls under the capabilities of the Code Assistant, as it involves programming.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


👨‍💼 Agent used: Code Assistant
🔄 Handoff occurred: Main Agent → Code Assistant
📝 Reason: The user is requesting help with writing a Python function, which falls under the capabilities of the Code Assistant, as it involves programming.
🤖 Response: Sure! Here's a simple Python function to calculate Fibonacci numbers using an iterative approach:

```python
def fibonacci(n):
    if n <= 0:
        return "Input should be a positive integer"
    el...
📊 Session info: 3 messages, Active: Code Assistant
--------------------------------------------------------------------------------

--- Step 2: User Query ---
User: Can you optimize this for better performance?
🤖 Currently active: Code Assistant
🤖 Current active agent: Code Assistant


...

🔄 Code Assistant handing off to Main Agent
📝 Handoff reason: The task involves data analysis, which may require expertise beyond programming, such as statistical or business analysis.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🎯 Main Agent routing to: Data Analyst
💭 Reasoning: The user is requesting an analysis of sales data, which falls under the expertise of the Data Analyst agent, who specializes in data analysis, statistics, and visualization.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


👨‍💼 Agent used: Data Analyst
🔄 Handoff occurred: Main Agent → Data Analyst
📝 Reason: The user is requesting an analysis of sales data, which falls under the expertise of the Data Analyst agent, who specializes in data analysis, statistics, and visualization.
🤖 Response: I can certainly help with analyzing sales data. Please provide the data set or describe the specific aspects you'd like to analyze, such as trends, patterns, or specific metrics. Additionally, let me ...
📊 Session info: 19 messages, Active: Data Analyst
--------------------------------------------------------------------------------

🎉 Conversation flow demonstration complete!

Key Features Demonstrated:
✅ Direct agent-to-user interaction
✅ Seamless handoffs between agents
✅ Context preservation across handoffs
✅ Intelligent routing based on query content
✅ Agents can continue conversations in their domain
✅ Automatic handoff when expertise is needed elsewhere
